In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Question 12:

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two D.P?

In [3]:
#Firstly, normalizing the dataset
def normalize_data(data):
    from sklearn.preprocessing import MinMaxScaler as mms
    scaler = mms()
    return pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

data_for_norm = data[['T2','T6']]#The data to normalize
norm_data = normalize_data(data_for_norm)

In [4]:
from sklearn.linear_model import LinearRegression as lr
model = lr()
X = np.array(norm_data['T2']).reshape(norm_data.shape[0],1)
y = norm_data['T6']
model.fit(X,y)
pred_values = model.predict(X)
from sklearn.metrics import r2_score
round(r2_score(pred_values,y),2)

0.44

## DATASET INSTRUCTION: <br>
### To answer the questions below, you will need to normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set.

In [5]:
#Firstly, normalizing the dataset
data_for_norm = data.drop(columns=['date','lights'])
norm_data = normalize_data(data_for_norm)

In [6]:
#Defining the features and labels
def X_and_y(data,label_name_list):
    return data.drop(columns=label_name_list), data[label_name_list]
features_df, label_df = X_and_y(norm_data,['Appliances'])

In [7]:
#spliting the data
def split_data(X,y,train_fraction,random_state):
    from sklearn.model_selection import train_test_split as tts
    return tts(X,y,test_size=1-train_fraction,random_state=random_state)
X_train,X_test,y_train,y_test = split_data(features_df,label_df,random_state=42,train_fraction=0.7,)

In [8]:
#Creating the model to be used for the questions
def use_model(model_name,alpha,dp):
    linear_model=getattr(__import__('sklearn'),'linear_model')
    curr_method=getattr(linear_model,model_name)
    try:
        lin_model=curr_method(alpha=alpha)
    except:
        lin_model=curr_method()
    lin_model.fit(X_train,y_train)
    
    y_pred = lin_model.predict(X_test).reshape(X_test.shape[0],1)
    
    from sklearn.metrics import mean_absolute_error as mae_
    from sklearn.metrics import mean_squared_error as mse_
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error as mse_
    
    mae = round(mae_(y_pred,y_test),dp)
    rmse = round(np.sqrt(mse_(y_test,y_pred)),dp)
    r2s = round(r2_score(y_test,y_pred),dp)
    rss = round(np.sum(np.square(y_test-y_pred)).iloc[0],dp)
    
    weights = pd.Series(lin_model.coef_.reshape(X_train.shape[1],),X_train.columns).sort_values()
    
    return {'model':str(lin_model),'mae':mae,'rss':rss,'rmse':rmse,'r2s':r2s}, weights#lin_model.coef_

def gen_answer(model_name, perf_metric='mae', dp=3, alpha=0.0, get_weights=False):
    try:
        result = use_model(model_name,dp=dp,alpha=alpha)[get_weights][perf_metric]
    except:
        result = use_model(model_name,dp=dp,alpha=alpha)[get_weights]
    
    return result

## Question 13:
<br>
What is the Mean Absolute Error (in two decimal places)?

In [9]:
gen_answer('LinearRegression', perf_metric='mae',dp=2)

0.05

## Question 14:
<br>
What is the Residual Sum of Squares (in two decimal places)?

In [10]:
gen_answer('LinearRegression', perf_metric='rss',dp=2)

45.35

## Question 15:
<br>
What is the Root Mean Squared Error (in three decimal places)?

In [11]:
gen_answer('LinearRegression', perf_metric='rmse',dp=3)

0.088

## Question 16:
<br>
What is the Coefficient of Determination (in two decimal places)?

In [12]:
#Note: Coefficient of Determination is the same as R-Square
gen_answer('LinearRegression', perf_metric='r2s',dp=2)

0.15

## Question 17:
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [13]:
weights = gen_answer('LinearRegression', get_weights=True)

#Since it is sorted in ascending order,
weights.keys()[0], weights.keys()[-1]#lowet and highest are:

('RH_2', 'RH_1')

## Question 18:
<br>
Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [14]:
#We checks if the scores are different
gen_answer('Ridge', perf_metric='rmse', alpha=0.4,dp=3) != gen_answer('LinearRegression', perf_metric='rmse', dp=3)

False

## Question 19:
<br>
Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [15]:
weights = gen_answer('Lasso',alpha=0.001, get_weights=True)
#Sum all non-zero weights
(weights!=0).sum()

4

##  Question 20:
<br>
What is the new RMSE with the Lasso Regression (in 3 decimal places)?

In [16]:
gen_answer('Lasso',alpha=0.001, perf_metric='rmse',dp=3)

0.094